In [1]:
import gensim
from nltk import word_tokenize, sent_tokenize
import multiprocessing
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from collections import Counter
from statistics import mean

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('clean_tweets_update.csv', usecols=['tweets', 'pos_lems'])

In [4]:
target_df = pd.read_csv('train.csv', usecols=['target'])
target = list(target_df.target)

In [5]:
print(len(target))

7613


In [6]:
train_df, test_df = df.iloc[:len(target_df), :], df.iloc[len(target_df):, :] 
train_df['target'] = target
train_df.drop_duplicates(subset=['pos_lems'], keep='last', inplace=True)

In [7]:
train_length = len(train_df)
print(train_length)

6851


In [8]:
test_length = len(test_df)
print(test_length)

3263


In [9]:
texts = [train_df.pos_lems, test_df.pos_lems]
tweets = pd.concat(texts)
len(tweets)

10114

In [10]:
train_tweets = train_df.tweets.replace('at_user', '')
train_tweets = train_tweets.replace('url_here', '')
test_tweets = test_df.tweets.replace('at_user', '')
test_tweets = test_tweets.replace('url_here', '')

In [16]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop.append('url_here')
stop.append('at_user')
stop.remove('not')
def tokenize_text(text):
    tokens = []
    for sent in sent_tokenize(text):
        for word in word_tokenize(sent):
            if len(word)>1 and word not in stop:
                tokens.append(word.lower())
    return tokens

In [17]:
train = train_tweets.apply(tokenize_text)
test = test_tweets.apply(tokenize_text)

## pretrained word_vectors model

In [18]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import keras

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout, Bidirectional
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

In [40]:
X_train, X_test, y_train, y_test = train_test_split(train, train_df.target, test_size=0.2, random_state=88)

train_df.lem = train_df.lem.replace('at_user', '')
test_df.lem = test_df.lem.replace('at_user', '')
train_df.lem = train_df.lem.replace('url_here', '')
test_df.lem = test_df.lem.replace('url_here', '')

In [41]:
from collections import Counter
from statistics import mean

word_dict = Counter(test.sum())
# count number of words in corpus
num_words_test = sum(word_dict[w] for w in word_dict)
print(f'There are {num_words_test} words in the unseen dataset')

# count number of unique words in corpus
word_count_sorted = [(value, key) for key, value in word_dict.items()]
word_count_sorted.sort()
vocab_size_test = len(word_count_sorted)
print(f'There are {vocab_size_test} unique words in the unseen dataset')
print("max len of training tweets",max([len(x) for x in test]))
print("min len of training tweets",min([len(x) for x in test]))
print("mean len of training tweets", mean([len(x) for x in test]))


There are 29205 words in the unseen dataset
There are 6902 unique words in the unseen dataset
max len of training tweets 22
min len of training tweets 0
mean len of training tweets 8.950352436408213


In [42]:
word_dict = Counter(X_train.sum())
# count number of words in corpus
num_words = sum(word_dict[w] for w in word_dict)
print(f'There are {num_words} words in the training dataset')

# count number of unique words in corpus
word_count_sorted = [(value, key) for key, value in word_dict.items()]
word_count_sorted.sort()
vocab_size = len(word_count_sorted)
maxlen = max([len(x) for x in X_train])
print(f'There are {vocab_size} unique words in the training dataset')
print("max len of training tweets",maxlen)
print("max len of test tweets",max([len(x) for x in X_test]))
print("min len of training tweets",min([len(x) for x in X_train]))
print("min len of test tweets",min([len(x) for x in X_test]))
print("mean len of training tweets", mean([len(x) for x in X_train]))
print("mean len of test tweets",mean([len(x) for x in X_test]))

There are 48418 words in the training dataset
There are 9581 unique words in the training dataset
max len of training tweets 24
max len of test tweets 20
min len of training tweets 1
min len of test tweets 1
mean len of training tweets 8.835401459854015
mean len of test tweets 8.733041575492342


# embedding size : 50

In [43]:
# load the word embedding file from local
embeddings_index = dict()
f = open('utils//glove.6B.50d.txt', encoding="utf8")
# iterate through text, splitting, obtain index & float32 for words
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
# close embedding file from local to save on memory
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [44]:
tokenizer = text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)
tokenized_train = tokenizer.texts_to_sequences(X_train)
x_train = np.sqrt(sequence.pad_sequences(tokenized_train, maxlen=maxlen))

tokenized_test = tokenizer.texts_to_sequences(X_test)
x_test = np.sqrt(sequence.pad_sequences(tokenized_test, maxlen=maxlen))

In [45]:
x_test

array([[ 0.        ,  0.        ,  0.        , ...,  5.65685425,
         6.4807407 , 16.673332  ],
       [ 0.        ,  0.        ,  0.        , ..., 18.49324201,
        13.3041347 ,  9.69535971],
       [ 0.        ,  0.        ,  0.        , ..., 64.        ,
        17.2626765 , 12.84523258],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  3.87298335,
         5.        , 14.73091986],
       [ 0.        ,  0.        ,  0.        , ...,  3.74165739,
        14.14213562, 53.08483776],
       [ 0.        ,  0.        ,  0.        , ..., 26.94438717,
        33.85262176, 62.84106937]])

In [46]:
tokenized_unseen_test = tokenizer.texts_to_sequences(test)
unseen_test = np.sqrt(sequence.pad_sequences(tokenized_unseen_test, maxlen=maxlen))

In [47]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
n_words = num_words
#change below line if computing normal stats is too slow
embedding_matrix = np.random.normal(emb_mean, emb_std,(n_words, embed_size))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

### callbacks

In [48]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.01, min_lr=0.05)
early_stop = EarlyStopping(monitor='loss', min_delta=0.01, patience=5, verbose=1)

In [52]:
batch_size = 300
epochs = 200
embed_size = 50

#### adam

In [53]:
#Defining Neural Network
model = Sequential()
#Non-trainable embedding layer
model.add(Embedding(n_words, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
#LSTM 
model.add(LSTM(256 , return_sequences = True , recurrent_dropout = 0.6 , dropout = 0.5))
model.add(LSTM(128, return_sequences = True , recurrent_dropout = 0.4 , dropout = 0.4))
model.add(LSTM(64, return_sequences = True , recurrent_dropout = 0.2 , dropout = 0.3))
model.add(LSTM(32, recurrent_dropout = 0.15 , dropout = 0.25))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [54]:
history = model.fit(x_train, y_train, batch_size = batch_size , validation_data = (x_test,y_test) , epochs = epochs , callbacks = [learning_rate_reduction])

Epoch 1/200
19/19 [==============================] - 12s 614ms/step - loss: 0.6731 - accuracy: 0.5874 - val_loss: 0.6554 - val_accuracy: 0.6010
Epoch 2/200
19/19 [==============================] - 11s 570ms/step - loss: 0.6645 - accuracy: 0.5907 - val_loss: 0.6554 - val_accuracy: 0.6010
Epoch 3/200
19/19 [==============================] - 11s 567ms/step - loss: 0.6646 - accuracy: 0.5905 - val_loss: 0.6544 - val_accuracy: 0.6010
Epoch 4/200
19/19 [==============================] - 11s 566ms/step - loss: 0.6629 - accuracy: 0.5907 - val_loss: 0.6540 - val_accuracy: 0.6010
Epoch 5/200
19/19 [==============================] - 11s 572ms/step - loss: 0.6613 - accuracy: 0.5909 - val_loss: 0.6606 - val_accuracy: 0.6010
Epoch 6/200
19/19 [==============================] - 11s 570ms/step - loss: 0.6624 - accuracy: 0.5903 - val_loss: 0.6540 - val_accuracy: 0.6010
Epoch 7/200
19/19 [==============================] - 11s 569ms/step - loss: 0.6609 - accuracy: 0.5905 - val_loss: 0.6536 - val_accuracy:

19/19 [==============================] - 11s 566ms/step - loss: 0.5966 - accuracy: 0.6797 - val_loss: 0.6309 - val_accuracy: 0.6565
Epoch 58/200
19/19 [==============================] - 11s 565ms/step - loss: 0.6018 - accuracy: 0.6679 - val_loss: 0.6360 - val_accuracy: 0.6528
Epoch 59/200
19/19 [==============================] - 11s 565ms/step - loss: 0.5908 - accuracy: 0.6818 - val_loss: 0.6465 - val_accuracy: 0.6411
Epoch 60/200
19/19 [==============================] - 11s 565ms/step - loss: 0.5910 - accuracy: 0.6819 - val_loss: 0.6272 - val_accuracy: 0.6528
Epoch 61/200
19/19 [==============================] - 11s 560ms/step - loss: 0.5883 - accuracy: 0.6816 - val_loss: 0.6326 - val_accuracy: 0.6389
Epoch 62/200
19/19 [==============================] - 11s 563ms/step - loss: 0.5826 - accuracy: 0.6891 - val_loss: 0.6215 - val_accuracy: 0.6579
Epoch 63/200
19/19 [==============================] - 11s 564ms/step - loss: 0.5802 - accuracy: 0.6925 - val_loss: 0.6363 - val_accuracy: 0.641

19/19 [==============================] - 11s 564ms/step - loss: 0.4315 - accuracy: 0.7942 - val_loss: 0.8234 - val_accuracy: 0.6236
Epoch 114/200
19/19 [==============================] - 11s 564ms/step - loss: 0.4211 - accuracy: 0.8004 - val_loss: 0.7660 - val_accuracy: 0.6397
Epoch 115/200
19/19 [==============================] - 11s 565ms/step - loss: 0.4380 - accuracy: 0.7916 - val_loss: 0.8049 - val_accuracy: 0.6346
Epoch 116/200
19/19 [==============================] - 11s 565ms/step - loss: 0.4071 - accuracy: 0.8100 - val_loss: 0.8220 - val_accuracy: 0.6455
Epoch 117/200
19/19 [==============================] - 11s 562ms/step - loss: 0.4215 - accuracy: 0.8029 - val_loss: 0.7945 - val_accuracy: 0.6404
Epoch 118/200
19/19 [==============================] - 11s 565ms/step - loss: 0.4085 - accuracy: 0.8068 - val_loss: 0.7994 - val_accuracy: 0.6389
Epoch 119/200
19/19 [==============================] - 11s 566ms/step - loss: 0.4116 - accuracy: 0.8058 - val_loss: 0.8247 - val_accuracy:

19/19 [==============================] - 10s 536ms/step - loss: 0.2785 - accuracy: 0.8801 - val_loss: 1.0571 - val_accuracy: 0.6222
Epoch 170/200
19/19 [==============================] - 10s 535ms/step - loss: 0.2872 - accuracy: 0.8743 - val_loss: 1.0571 - val_accuracy: 0.6083
Epoch 171/200
19/19 [==============================] - 10s 537ms/step - loss: 0.2656 - accuracy: 0.8870 - val_loss: 1.1146 - val_accuracy: 0.6134
Epoch 172/200
19/19 [==============================] - 10s 536ms/step - loss: 0.2730 - accuracy: 0.8785 - val_loss: 1.0484 - val_accuracy: 0.6324
Epoch 173/200
19/19 [==============================] - 10s 537ms/step - loss: 0.2704 - accuracy: 0.8859 - val_loss: 1.1254 - val_accuracy: 0.6229
Epoch 174/200
19/19 [==============================] - 10s 537ms/step - loss: 0.2906 - accuracy: 0.8754 - val_loss: 1.0512 - val_accuracy: 0.6127
Epoch 175/200
19/19 [==============================] - 10s 537ms/step - loss: 0.2669 - accuracy: 0.8841 - val_loss: 1.1082 - val_accuracy:

In [55]:
ynew = np.argmax(model.predict(unseen_test), axis=-1)
submission = pd.read_csv('test.csv', usecols=['id'])
submission['target'] = ynew
submission.to_csv('predictions.csv')

#### rmsprop

In [46]:
#Defining Neural Network
model = Sequential()
#Non-trainable embedding layer
model.add(Embedding(n_words, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
#LSTM 
model.add(LSTM(256 , return_sequences = True , recurrent_dropout = 0.5 , dropout = 0.5))
model.add(LSTM(128, recurrent_dropout = 0.6 , dropout = 0.3))
#model.add(Dense(32, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
#model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.RMSprop(), loss='binary_crossentropy', metrics=['accuracy'])

In [47]:
history = model.fit(x_train, y_train, batch_size = batch_size , validation_data = (x_test,y_test) , epochs = epochs)

Epoch 1/100
19/19 [==============================] - 7s 351ms/step - loss: 0.6059 - accuracy: 0.6619 - val_loss: 0.4849 - val_accuracy: 0.7826
Epoch 2/100
19/19 [==============================] - 7s 357ms/step - loss: 0.5142 - accuracy: 0.7588 - val_loss: 0.4663 - val_accuracy: 0.7921
Epoch 3/100
19/19 [==============================] - 7s 357ms/step - loss: 0.5042 - accuracy: 0.7693 - val_loss: 0.4729 - val_accuracy: 0.7885
Epoch 4/100
19/19 [==============================] - 7s 383ms/step - loss: 0.4946 - accuracy: 0.7723 - val_loss: 0.4796 - val_accuracy: 0.7826
Epoch 5/100
19/19 [==============================] - 7s 367ms/step - loss: 0.4941 - accuracy: 0.7692 - val_loss: 0.4648 - val_accuracy: 0.8016
Epoch 6/100
19/19 [==============================] - 7s 369ms/step - loss: 0.4946 - accuracy: 0.7699 - val_loss: 0.4468 - val_accuracy: 0.8053
Epoch 7/100
19/19 [==============================] - 7s 372ms/step - loss: 0.4857 - accuracy: 0.7712 - val_loss: 0.4596 - val_accuracy: 0.8001

Epoch 58/100
19/19 [==============================] - 7s 393ms/step - loss: 0.3391 - accuracy: 0.8544 - val_loss: 0.4547 - val_accuracy: 0.8082
Epoch 59/100
19/19 [==============================] - 8s 396ms/step - loss: 0.3349 - accuracy: 0.8529 - val_loss: 0.4806 - val_accuracy: 0.8118
Epoch 60/100
19/19 [==============================] - 8s 396ms/step - loss: 0.3355 - accuracy: 0.8527 - val_loss: 0.5119 - val_accuracy: 0.8031
Epoch 61/100
19/19 [==============================] - 8s 400ms/step - loss: 0.3322 - accuracy: 0.8595 - val_loss: 0.5431 - val_accuracy: 0.7929
Epoch 62/100
19/19 [==============================] - 8s 398ms/step - loss: 0.3232 - accuracy: 0.8586 - val_loss: 0.4816 - val_accuracy: 0.8082
Epoch 63/100
19/19 [==============================] - 8s 399ms/step - loss: 0.3251 - accuracy: 0.8615 - val_loss: 0.4883 - val_accuracy: 0.8125
Epoch 64/100
19/19 [==============================] - 8s 398ms/step - loss: 0.3187 - accuracy: 0.8646 - val_loss: 0.4965 - val_accuracy:

In [694]:
ynew = model.predict_classes(unseen_test)
submission = pd.read_csv('test.csv', usecols=['id'])
submission['target'] = ynew
submission.to_csv('predictions1.csv')

# embedding size : 100

#### load embeddings from local

In [64]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('utils//glove.6B.100d.txt', encoding="utf8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [68]:
# fit embeddings to data
X_train, X_test, y_train, y_test = train_test_split(train, train_df.target, test_size=0.2, random_state=88)
tokenizer = text.Tokenizer(num_words=vocab_size) # vocab size of training data
tokenizer.fit_on_texts(X_train)
tokenized_train = tokenizer.texts_to_sequences(X_train)
x_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

tokenized_test = tokenizer.texts_to_sequences(X_test)
x_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [70]:
tokenized_unseen_test = tokenizer.texts_to_sequences(test)
unseen_test = sequence.pad_sequences(tokenized_unseen_test, maxlen=30)

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
n_words = num_words
#change below line if computing normal stats is too slow
embedding_matrix = np.random.normal(emb_mean, emb_std,(n_words, embed_size))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [67]:
maxlen

28

In [53]:
batch_size = 300
epochs = 150
embed_size = 100

#### adam

In [69]:
#Defining Neural Network
model = Sequential()
#Non-trainable embedding layer
model.add(Embedding(n_words, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
#LSTM 
#model.add(LSTM(256, return_sequences = True, recurrent_dropout = 0.4 , dropout = 0.4))
model.add(Bidirectional(LSTM(128, return_sequences = True , recurrent_dropout = 0.5 , dropout = 0.5)))
#model.add(LSTM(64, return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(64, dropout = 0.4))
#model.add(Dense(25, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
#model.add(Dense(8, activation = 'relu'))
#model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy']) 

In [70]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 24, 50)            2420900   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 24, 256)           183296    
_________________________________________________________________
lstm_30 (LSTM)               (None, 64)                82176     
_________________________________________________________________
dense_24 (Dense)             (None, 16)                1040      
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 17        
Total params: 2,687,429
Trainable params: 266,529
Non-trainable params: 2,420,900
_________________________________________________________________


In [ ]:
history = model.fit(x_train, y_train, batch_size = 300, validation_data = (x_test,y_test) , epochs = 150 , callbacks = [learning_rate_reduction], shuffle=True)

Epoch 1/150
19/19 [==============================] - 7s 376ms/step - loss: 0.5793 - accuracy: 0.6914 - val_loss: 0.6324 - val_accuracy: 0.6477
Epoch 2/150
19/19 [==============================] - 7s 369ms/step - loss: 0.5718 - accuracy: 0.7047 - val_loss: 0.6382 - val_accuracy: 0.6499
Epoch 3/150
19/19 [==============================] - 7s 374ms/step - loss: 0.5696 - accuracy: 0.6998 - val_loss: 0.6443 - val_accuracy: 0.6419
Epoch 4/150
19/19 [==============================] - 7s 365ms/step - loss: 0.5672 - accuracy: 0.6971 - val_loss: 0.6391 - val_accuracy: 0.6492
Epoch 5/150
19/19 [==============================] - 7s 373ms/step - loss: 0.5645 - accuracy: 0.7024 - val_loss: 0.6498 - val_accuracy: 0.6419
Epoch 6/150
19/19 [==============================] - 7s 366ms/step - loss: 0.5607 - accuracy: 0.7106 - val_loss: 0.6503 - val_accuracy: 0.6397
Epoch 7/150
19/19 [==============================] - 7s 374ms/step - loss: 0.5573 - accuracy: 0.7124 - val_loss: 0.6451 - val_accuracy: 0.6506

In [ ]:
ynew = model.predict_classes(unseen_test)
submission = pd.read_csv('test.csv', usecols=['id'])
submission['target'] = ynew
submission.to_csv('predictions2.csv')

In [ ]:
### rmsprop

In [56]:
#Defining Neural Network
model = Sequential()
#Non-trainable embedding layer
model.add(Embedding(n_words, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
#LSTM 
#LSTM 
model.add(LSTM(256, return_sequences = True, recurrent_dropout = 0.4 , dropout = 0.4))
model.add(LSTM(128, return_sequences = True , recurrent_dropout = 0.35 , dropout = 0.35))
model.add(LSTM(64, return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(32, dropout = 0.4))
#model.add(Dense(25, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
#model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.RMSprop(), loss='binary_crossentropy', metrics=['accuracy']) 

In [57]:
history = model.fit(x_train, y_train, batch_size = batch_size , validation_data = (x_test,y_test) , epochs = epochs )

Epoch 1/150
19/19 [==============================] - 20s 1s/step - loss: 0.6718 - accuracy: 0.5949 - val_loss: 0.5992 - val_accuracy: 0.6842
Epoch 2/150
19/19 [==============================] - 19s 999ms/step - loss: 0.5806 - accuracy: 0.7157 - val_loss: 0.5106 - val_accuracy: 0.7834
Epoch 3/150
19/19 [==============================] - 19s 989ms/step - loss: 0.5252 - accuracy: 0.7599 - val_loss: 0.5206 - val_accuracy: 0.7360
Epoch 4/150
19/19 [==============================] - 19s 987ms/step - loss: 0.5095 - accuracy: 0.7670 - val_loss: 0.4778 - val_accuracy: 0.7863
Epoch 5/150
19/19 [==============================] - 19s 992ms/step - loss: 0.4952 - accuracy: 0.7724 - val_loss: 0.4665 - val_accuracy: 0.7921
Epoch 6/150
19/19 [==============================] - 19s 999ms/step - loss: 0.4909 - accuracy: 0.7772 - val_loss: 0.4667 - val_accuracy: 0.7965
Epoch 7/150
19/19 [==============================] - 19s 995ms/step - loss: 0.4795 - accuracy: 0.7838 - val_loss: 0.4987 - val_accuracy: 0.

Epoch 58/150
19/19 [==============================] - 19s 994ms/step - loss: 0.2426 - accuracy: 0.8982 - val_loss: 0.5955 - val_accuracy: 0.8162
Epoch 59/150
19/19 [==============================] - 19s 996ms/step - loss: 0.2329 - accuracy: 0.9075 - val_loss: 0.5838 - val_accuracy: 0.7892
Epoch 60/150
19/19 [==============================] - 19s 990ms/step - loss: 0.2395 - accuracy: 0.9027 - val_loss: 0.5701 - val_accuracy: 0.7929
Epoch 61/150
19/19 [==============================] - 19s 991ms/step - loss: 0.2221 - accuracy: 0.9095 - val_loss: 0.6299 - val_accuracy: 0.8031
Epoch 62/150
19/19 [==============================] - 19s 990ms/step - loss: 0.2355 - accuracy: 0.9046 - val_loss: 0.5711 - val_accuracy: 0.7929
Epoch 63/150
19/19 [==============================] - 19s 989ms/step - loss: 0.2220 - accuracy: 0.9091 - val_loss: 0.6319 - val_accuracy: 0.7710
Epoch 64/150
19/19 [==============================] - 19s 986ms/step - loss: 0.2109 - accuracy: 0.9201 - val_loss: 0.6219 - val_ac

In [58]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 27, 100)           6306000   
_________________________________________________________________
lstm_14 (LSTM)               (None, 27, 256)           365568    
_________________________________________________________________
lstm_15 (LSTM)               (None, 27, 128)           197120    
_________________________________________________________________
lstm_16 (LSTM)               (None, 27, 64)            49408     
_________________________________________________________________
lstm_17 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dense_11 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_12 (Dense)             (None, 8)                

In [705]:
ynew = model.predict_classes(unseen_test)
submission = pd.read_csv('test.csv', usecols=['id'])
submission['target'] = ynew
submission.to_csv('predictions3.csv')